In [11]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('mm_web_dump_2020-03-22.sqlite')

#### Load team stats from database

In [12]:
all_stats = pd.read_sql_query('''
SELECT Team_Stats1.*, School.name, Season.year, Rank.tourn_rank, 
       TS2.pace, TS2.ast_pct, TS2.blk_pct, TS2.tov_pct, 
       OS1.opp_pts, OS1.opp_fg3, OS1.opp_fta, OS1.opp_off_reb, OS1.opp_def_reb, 
       OS1.opp_ast, OS1.opp_stl, OS1.opp_blk, OS1.opp_tov, 
       OS2.opp_blk_pct, OS2.opp_tov_pct
FROM Team_Stats1
INNER JOIN School ON Team_Stats1.school_id = School.id
INNER JOIN Season ON Team_Stats1.season_id = Season.id
INNER JOIN Rank 
  ON Team_Stats1.school_id = Rank.school_id AND Team_Stats1.season_id = Rank.season_id
INNER JOIN Team_Stats2 AS TS2 
  ON Team_Stats1.school_id = TS2.school_id AND Team_Stats1.season_id = TS2.season_id
INNER JOIN Opp_Stats1 AS OS1 
  ON Team_Stats1.school_id = OS1.school_id AND Team_Stats1.season_id = OS1.season_id
INNER JOIN Opp_Stats2 AS OS2 
  ON Team_Stats1.school_id = OS2.school_id AND Team_Stats1.season_id = OS2.season_id
''', conn)
# all_stats.to_csv('all_stats.csv')

#### Normalize some variables to average values over 40 minutes of play

In [4]:
var_list = ['pts','opp_pts','fg3','opp_fg3','ft','opp_fta','off_reb','opp_off_reb','def_reb','opp_def_reb', 
            'ast','opp_ast','stl','opp_stl','blk','opp_blk','tov','opp_tov']
for var in var_list:
    all_stats[var] = all_stats[var] / all_stats['minutes'] * 40

#### Engineer some new features

In [5]:
all_stats['score_margin'] = all_stats['pts'] - all_stats['opp_pts']
all_stats['or_pct'] = all_stats['off_reb'] / (all_stats['off_reb'] + all_stats['opp_def_reb'])
all_stats['dr_pct'] = all_stats['def_reb'] / (all_stats['def_reb'] + all_stats['opp_off_reb'])
all_stats['tr_pct'] = (all_stats['off_reb'] + all_stats['def_reb']) /  \
                      (all_stats['off_reb'] + all_stats['def_reb'] + all_stats['opp_off_reb'] + all_stats['opp_def_reb'])
all_stats = all_stats.drop(columns=['name','year','minutes','pts','opp_pts','off_reb','opp_off_reb','def_reb','opp_def_reb'])
all_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677 entries, 0 to 676
Data columns (total 26 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   school_id     677 non-null    int64  
 1   season_id     677 non-null    int64  
 2   srs           677 non-null    float64
 3   fg3           677 non-null    float64
 4   ft            677 non-null    float64
 5   ast           677 non-null    float64
 6   stl           677 non-null    float64
 7   blk           677 non-null    float64
 8   tov           677 non-null    float64
 9   tourn_rank    677 non-null    int64  
 10  pace          677 non-null    float64
 11  ast_pct       677 non-null    float64
 12  blk_pct       677 non-null    float64
 13  tov_pct       677 non-null    float64
 14  opp_fg3       677 non-null    float64
 15  opp_fta       677 non-null    float64
 16  opp_ast       677 non-null    float64
 17  opp_stl       677 non-null    float64
 18  opp_blk       677 non-null    

#### Load tournament matchups and results from database

In [6]:
matchups = pd.read_sql_query("SELECT * FROM Tournament", conn)
# invert matchups to create complete set
m_inverse = matchups.copy()
m_inverse = m_inverse.rename(columns={"school_id1": "school_id2", "school_id2": "school_id1", 
                                      "score1": "score2", "score2": "score1"})
matchups = matchups.append(m_inverse)

#### Create response variables for regression and classification models

In [7]:
matchups['result_regr'] = matchups['score1'] - matchups['score2']
matchups['result_class'] = matchups['result_regr'].apply(lambda x: 1 if x>0 else 0)
matchups = matchups.drop(columns=['score1','score2'])
matchups.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1334 entries, 0 to 666
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   season_id     1334 non-null   int64
 1   school_id1    1334 non-null   int64
 2   school_id2    1334 non-null   int64
 3   result_regr   1334 non-null   int64
 4   result_class  1334 non-null   int64
dtypes: int64(5)
memory usage: 62.5 KB


#### Merge matchups with team stats

In [8]:
matchups = matchups.merge(all_stats, how='left', left_on=['season_id','school_id1'], 
                          right_on=['season_id','school_id'])
matchups = matchups.merge(all_stats, how='left', left_on=['season_id','school_id2'], 
                          right_on=['season_id','school_id'], suffixes=[None,'_2'])
matchups.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1334 entries, 0 to 1333
Data columns (total 55 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   season_id       1334 non-null   int64  
 1   school_id1      1334 non-null   int64  
 2   school_id2      1334 non-null   int64  
 3   result_regr     1334 non-null   int64  
 4   result_class    1334 non-null   int64  
 5   school_id       1334 non-null   int64  
 6   srs             1334 non-null   float64
 7   fg3             1334 non-null   float64
 8   ft              1334 non-null   float64
 9   ast             1334 non-null   float64
 10  stl             1334 non-null   float64
 11  blk             1334 non-null   float64
 12  tov             1334 non-null   float64
 13  tourn_rank      1334 non-null   int64  
 14  pace            1334 non-null   float64
 15  ast_pct         1334 non-null   float64
 16  blk_pct         1334 non-null   float64
 17  tov_pct         1334 non-null   f

#### Create dataset for modeling

In [9]:
mm_data = matchups[['result_regr','result_class']].copy()
mm_data['srs_diff'] = matchups['srs'] - matchups['srs_2']
mm_data['rank_diff'] = matchups['tourn_rank'] - matchups['tourn_rank_2']
mm_data['margin_diff'] = matchups['score_margin'] - matchups['score_margin_2']
mm_data['pace_diff'] = matchups['pace'] - matchups['pace_2']

mm_data['fg3_diff_a'] = matchups['fg3'] - matchups['fg3_2']
mm_data['fg3_diff_b'] = (matchups['fg3'] + matchups['opp_fg3_2']) - (matchups['fg3_2'] + matchups['opp_fg3'])
mm_data['opp_fg3_diff'] = matchups['opp_fg3'] - matchups['opp_fg3_2']

mm_data['ft_diff_a'] = matchups['ft'] - matchups['ft_2']
mm_data['ft_diff_b'] = (matchups['ft'] + matchups['opp_fta_2']) - (matchups['ft_2'] + matchups['opp_fta'])
mm_data['opp_fta_diff'] = matchups['opp_fta'] - matchups['opp_fta_2']

mm_data['ast_diff'] = matchups['ast'] - matchups['ast_2']
mm_data['ast_pct_diff'] = matchups['ast_pct'] - matchups['ast_pct_2']
mm_data['opp_ast_diff'] = matchups['opp_ast'] - matchups['opp_ast_2']
mm_data['ast_tov_diff'] = matchups['ast']/matchups['tov'] - matchups['ast_2']/matchups['tov_2']

mm_data['stl_diff_a'] = matchups['stl'] - matchups['stl_2']
mm_data['stl_diff_b'] = (matchups['stl'] + matchups['opp_stl_2']) - (matchups['stl_2'] + matchups['opp_stl'])
mm_data['opp_stl_diff'] = matchups['opp_stl'] - matchups['opp_stl_2']
mm_data['stl_tov'] = (matchups['stl'] + matchups['tov_2']) - (matchups['stl_2'] + matchups['tov'])
mm_data['tov_diff_a'] = matchups['tov'] - matchups['tov_2']
mm_data['tov_diff_b'] = (matchups['opp_tov'] + matchups['tov_2']) - (matchups['opp_tov_2'] + matchups['tov'])
mm_data['opp_tov_diff'] = matchups['opp_tov'] - matchups['opp_tov_2']
mm_data['tov_pct_diff_a'] = matchups['tov_pct'] - matchups['tov_pct_2']
mm_data['tov_pct_diff_b'] = (matchups['opp_tov_pct'] + matchups['tov_pct_2']) - (matchups['opp_tov_pct_2'] + matchups['tov_pct'])
mm_data['opp_tov_pct_diff'] = matchups['opp_tov_pct'] - matchups['opp_tov_pct_2']

mm_data['blk_diff_a'] = matchups['blk'] - matchups['blk_2']
mm_data['blk_diff_b'] = (matchups['blk'] + matchups['opp_blk_2']) - (matchups['blk_2'] + matchups['opp_blk']) 
mm_data['opp_blk_diff'] = matchups['opp_blk'] - matchups['opp_blk_2']
mm_data['blk_pct_diff_a'] = matchups['blk_pct'] - matchups['blk_pct_2']
mm_data['blk_pct_diff_b'] = (matchups['blk_pct'] + matchups['opp_blk_pct_2']) - (matchups['blk_pct_2'] + matchups['opp_blk_pct']) 
mm_data['opp_blk_pct_diff'] = matchups['opp_blk_pct'] - matchups['opp_blk_pct_2']

mm_data['or_diff'] = matchups['or_pct'] - matchups['or_pct_2']
mm_data['dr_diff'] = matchups['dr_pct'] - matchups['dr_pct_2']
mm_data['or_dr'] = matchups['or_pct'] - matchups['dr_pct_2']
mm_data['dr_or'] = matchups['dr_pct'] - matchups['or_pct_2']
mm_data['tr_diff'] = matchups['tr_pct'] - matchups['tr_pct_2']

mm_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1334 entries, 0 to 1333
Data columns (total 37 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   result_regr       1334 non-null   int64  
 1   result_class      1334 non-null   int64  
 2   srs_diff          1334 non-null   float64
 3   rank_diff         1334 non-null   int64  
 4   margin_diff       1334 non-null   float64
 5   pace_diff         1334 non-null   float64
 6   fg3_diff_a        1334 non-null   float64
 7   fg3_diff_b        1334 non-null   float64
 8   opp_fg3_diff      1334 non-null   float64
 9   ft_diff_a         1334 non-null   float64
 10  ft_diff_b         1334 non-null   float64
 11  opp_fta_diff      1334 non-null   float64
 12  ast_diff          1334 non-null   float64
 13  ast_pct_diff      1334 non-null   float64
 14  opp_ast_diff      1334 non-null   float64
 15  ast_tov_diff      1334 non-null   float64
 16  stl_diff_a        1334 non-null   float64


In [10]:
mm_data.to_csv('mm_data.csv', index=False)